#### Forked from  notebook - `plot trend pred + CV-V2023`
###### - removed mos EDA + do CV/evaluation by Groups/series instead of by time


##### Publication trends forecasting outputs

* Train+val+test Data covers ~1960-2019.
    * Model/features optimized over 1980-2010

* Prediction 5 years (inclusive) in advance

* Groupwise split over data, by topic

* `y_raw` = Actual amount of publications (normalized out of total pubs that year in pubmed)
* `preds` = model predictions


In [18]:
import pandas as pd
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import cross_val_predict,TimeSeriesSplit, GroupKFold
from sklearn.ensemble import HistGradientBoostingRegressor 
from sklearn import linear_model
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV, ElasticNet,ElasticNetCV, Lasso
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.metrics import PredictionErrorDisplay
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
from sklearn.feature_selection import RFECV
import sklearn.metrics as metrics
pd.set_option('mode.use_inf_as_na', True)

In [19]:
FILTER_GROUP_SIZES = True

DIFF_TARGET = False#True#False#True # set target to diff vs 5 years ago. i.e tougher baseline
DIFF_TARGET_PCT = False#True#False#True
NUM_CV_FOLDS =  30#5#30 #

DO_RFE_FS = False#False#True

GET_DL_EMBEDS = True#False

In [20]:
def plot_var_preds(df,val:str="subthalamus",logScale=False):
#     df.loc[df["variable"].str.contains(val,case=False)][["y_raw","preds"]].plot(title=val)# "y_raw_predicted"
    if logScale:
        df.loc[df["variable"].str.lower()==val.lower()][["y_raw","preds"]].apply(np.log1p).plot(title=val)
    else:
        df.loc[df["variable"].str.lower()==val.lower()][["y_raw","preds"]].plot(title=val)
    
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('r2:', round(r2,3))
    print('MAE (Mean absolute error):', round(mean_absolute_error,3))
    print('Median absolute error:', round(median_absolute_error,3))
#     print('MSE: ', round(mse,3))
    print('RMSE:', round(np.sqrt(mse),3))
    print('explained_variance: ', round(explained_variance,3))    
    try:
        mape = metrics.mean_absolute_percentage_error(y_true, y_pred)
        print('mape:', round(mape,3))
    except:()
    try: 
        mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
        print('mean_squared_log_error:', round(mean_squared_log_error,3))
    except:()

        
def evaluate(model = CatBoostRegressor(cat_features= ["variable"],verbose=False,has_time=True),X = None,
            USE_TIME_SPLIT = False):
    """
    Assigns results of preictions to df_feat (global var version)
    https://stats.stackexchange.com/questions/495151/cross-validation-for-time-series-what-am-i-doing-wrong
    https://stackoverflow.com/questions/51597507/sklearn-timeseriessplit-error-keyerror-0-1-2-not-in-index
    """
    # train model on training dataset
    i = 0
    if USE_TIME_SPLIT:
        tscv = TimeSeriesSplit(NUM_CV_FOLDS) #10) # ORIG - 10
    else:
        tscv = GroupKFold(n_splits=10)
    
#     df_feat["preds"] = np.NaN # un commented - DAN - new
    for train_index, test_index in tscv.split(X): #df_feat):
        i +=1
#         print(i)
        if i%3==0: print(i)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#         X_train, X_test = X.index[train_index], X.index[test_index]#X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         model = CatBoostRegressor(iterations=500,
#                                   cat_features= ["variable"],#[0], #["variable"],
#                           verbose=False,has_time=True)
#         print("fit")
#         model.fit(X_train, y_train)
        model.fit(X_train, y_train,
#                   early_stopping_rounds=10,
#                   eval_set=(X_test, y_test),
                  #plot=(i%3==0),
#                  baseline=X_train["lag5"].values)#pd.concat([X_train["lag5"],X_test["lag5"]]).values)
                  )
#         print("doing pred")
        y_predict = model.predict(X_test)
        assert np.isnan(y_predict).max() == False
#         print("assigning pred")
#         df_feat.loc[test_index,"preds"] =y_predict
        df_feat.iloc[test_index,-1] =y_predict  ##
    print("\n Done")

def truncate(f2, n=1):
    '''Truncates/pads a float f to n decimal places without rounding'''
#     if f.isnumeric():
    try: 
        f = float(f2)
#         s = '{}'.format(f)
#         if 'e' in s or 'E' in s:
#             return '{0:.{1}f}'.format(f, n)
#         i, p, d = s.partition('.')
#         return '.'.join([i, (d+'0'*n)[:n]])
        return(str(round(float(f),2)))
#     else:
    except:
        return f2

#### plot (lagged?) correlation of inputs and target in raw data

In [21]:
df_targets = pd.read_csv("trends_v6.csv.gz",parse_dates=["Year","first","start"],infer_datetime_format=True)

#### NEW! Sort by date!
df_targets = df_targets.sort_values(["Year","variable"])
df_targets["target_5"] = df_targets.groupby("variable")["y_raw"].shift(-5) ## target in 5 years

# df_targets[["Year","variable","y_raw","target_5"]]
df_targets = df_targets.loc[(df_targets["Year"]> df_targets["start"]) & (df_targets["year_num"]>= 1979) ]
df_targets.drop(columns=[
#                          "pct_diff_lag6",
    "y_pct_bins",
#                          "y_pct_diff","year_num",
                         "lag1","lag2","lag3","lag4",
##ORIG: was commented out! be careful! #                          "diff_lag6","lag6",
#                          "pct_diff_lag5",
#                          "first","start"
            ],errors="ignore",inplace=True)
print(df_targets.shape)
df_targets["lag5_pct_new"] = 100*df_targets["diff_lag5"].div(df_targets["y_raw"])# pct diff vs 5 years ago

df_targets = df_targets.reset_index(drop=True)

df_targets.head(7)

(4798, 25)


,Year,variable,y_raw,reviews,research_review_diff,research_review_ratio,pct_diff,patent_count,patent_frac,patent_yearly_total,...,pat_div_research,year_num,y_diff,y_pct_diff,diff_lag5,diff_lag6,pct_diff_lag5,pct_diff_lag6,target_5,lag5_pct_new
0,1979-01-01,BLAST,103.5087,1.4179,103.5087,1.0000,-0.0271,87.0,4.5645,1906.0,...,0.8405,1979,-2.8860,-0.027125,63.1021,69.3282,0.182154,0.052524,150.1954,60.963088
1,1979-01-01,Brodmann area,2.1269,0.0000,-4.2538,-2.0000,0.9324,0.0,0.0000,1906.0,...,0.0000,1979,1.0263,0.932491,1.7016,2.1269,NaN,NaN,3.1422,80.003761
2,1979-01-01,Cingulate cortex,13.8248,0.0000,13.8248,1.0000,-0.0084,0.0,0.0000,1906.0,...,0.0000,1979,-0.1166,-0.008364,-2.3378,-6.9431,-0.221751,0.631850,25.1373,-16.910190
3,1979-01-01,DNA array,4.2538,0.3545,4.2538,1.0000,0.1595,0.0,0.0000,1906.0,...,0.0000,1979,0.5850,0.159453,3.4031,3.3885,NaN,NaN,13.1971,80.001411
4,1979-01-01,Dorsolateral prefrontal cortex,1.0634,0.0000,-40.0564,-37.6667,-0.5859,0.0,0.0000,1906.0,...,0.0000,1979,-1.5047,-0.585920,0.6381,0.6307,NaN,NaN,2.8279,60.005642
5,1979-01-01,EEG,719.9524,20.5599,699.0379,0.9710,0.0237,1.0,0.0525,1906.0,...,0.0014,1979,16.6468,0.023669,-226.4131,-321.0382,-0.090899,-0.016952,683.1065,-31.448343
6,1979-01-01,Epidemiology,3853.2162,153.1361,3853.2162,1.0000,0.0485,0.0,0.0000,1906.0,...,0.0000,1979,178.1969,0.048489,-510.6986,-559.9610,-0.011163,0.079809,4098.6388,-13.253827


In [22]:
df_targets["first"] = df_targets["first"].str.split("-",expand=True)[0].astype(int)
df_targets["start"] = pd.to_datetime(df_targets["start"]).dt.year.astype(int)
df_targets["start_sub_first"] = df_targets["start"].sub(df_targets["first"])
df_targets["year_sub_start"] = df_targets["year_num"].sub(df_targets["start"])
df_targets[["target_5","y_raw","reviews","first","start","start_sub_first","year_sub_start"]].corr().round(3)

,target_5,y_raw,reviews,first,start,start_sub_first,year_sub_start
target_5,1.000,0.987,0.902,-0.027,-0.223,-0.030,0.236
y_raw,0.987,1.000,0.905,-0.021,-0.229,-0.031,0.250
reviews,0.902,0.905,1.000,-0.021,-0.217,-0.028,0.281
first,-0.027,-0.021,-0.021,1.000,0.128,-0.975,-0.056
start,-0.223,-0.229,-0.217,0.128,1.000,0.098,-0.724
start_sub_first,-0.030,-0.031,-0.028,-0.975,0.098,1.000,-0.107
year_sub_start,0.236,0.250,0.281,-0.056,-0.724,-0.107,1.000


In [23]:
if FILTER_GROUP_SIZES:
    print(df_targets.shape[0])
    element_group_sizes = df_targets['variable'].groupby(df_targets['variable']).transform('count')
    df_targets = df_targets[element_group_sizes>6] # was 5, changed for CV results
    df_targets.reset_index(inplace=True,drop=True) # otherwise, error ?? 
    print(df_targets.shape[0])

if DIFF_TARGET_PCT:
    print("DIFF_TARGET_PCT")
    df_targets["target_5"] = 100*df_targets["target_5"].sub(df_targets["lag5"]).div(df_targets["y_raw"]).round(4) # alt, pct change target
#     df_targets["y_raw"] = 100*df_targets["y_raw"].div(df_targets["lag5"]).round(4) # alt, pct change target
elif DIFF_TARGET:
    print("DIFF_TARGET")
    df_targets["target_5"] = df_targets["target_5"].sub(df_targets["lag5"])
#     df_targets["y_raw"] = df_targets["y_raw"].sub(df_targets["lag5"]) # not needed? 
#     df_targets["y_raw"] = df_targets["y_raw"].div(df_targets["lag5"]) # alt, pct change target
if DIFF_TARGET_PCT or DIFF_TARGET:
    print("differencing of target applied")
#     df_targets.dropna(subset=["y_raw"],inplace=True)
    df_targets_orig_full = df_targets.copy() # new, keep copy of full data for getting future preds
    df_targets.dropna(subset=["target_5"],inplace=True) ## originally we dropped - keep these for predictions! 
    df_targets.reset_index(inplace=True,drop=True)
    display(df_targets[["target_5","y_raw"]].describe().round(3))
else:
    df_targets_orig_full = df_targets.copy()

4798
4790


In [24]:
if DIFF_TARGET_PCT:
    print("percent differenced target has some outliers (especially around start of a topic) and some extreme values (11,644)")
    ### extreme growth topics - subthalamus, rnai, MRI, fMRI, carbon nanotubes, perceptron - makes sense
    display(df_targets[df_targets["target_5"]>3000].sort_values(["target_5"],ascending=False).filter(["Year","variable","y_raw","target_5"],axis=1))


In [25]:
df_targets["target_5"].corr(df_targets['y_raw']).round(3)

0.987

### Baseline model - lag raw
* baseline model of input features, lag 5

In [26]:
# df_targets["target_5"] = df_targets.groupby("variable")["y_raw"].shift(-2)
df_targets[["Year","variable","y_raw","target_5"]]

,Year,variable,y_raw,target_5
0,1979-01-01,BLAST,103.5087,150.1954
1,1979-01-01,Brodmann area,2.1269,3.1422
2,1979-01-01,Cingulate cortex,13.8248,25.1373
3,1979-01-01,DNA array,4.2538,13.1971
4,1979-01-01,Dorsolateral prefrontal cortex,1.0634,2.8279
...,...,...,...,...
4785,2020-01-01,vaccine,1641.1449,NaN
4786,2020-01-01,viral therapy,1272.5626,NaN
4787,2020-01-01,vitamin C,186.8140,NaN
4788,2020-01-01,vitamin D,351.0454,NaN


In [27]:
# df_lagged = df_targets.dropna(subset=["target_5"],axis=0).copy()

df_lagged = df_targets.filter(['variable', 'reviews',
                              'patent_count', 'patent_frac','patent_yearly_total',
                               'year_num', 
"y_raw",                              ## ADdded: Here it is 5 befre?  NEW
                              "lag5_pct_new","pct_diff_lag5", ## also new - check not leak? 
                              'research_review_diff',
       'research_review_ratio', 'pct_diff', 
                              'review_research_ratio_2',
       'review_research_diff_2', 'pat_div_research',
        'y_diff','y_pct_diff', 
        'target_5']).reset_index(drop=True)

num_cols = list(df_lagged.select_dtypes("number").drop(columns=['target_5',
#                                                                 "y_raw", ### Keeep? NEW
                                                                "y_raw_predicted","preds"],errors="ignore").columns)
df_lagged = df_lagged.filter(["variable","target_5"]+num_cols,axis=1)
mask = ~df_lagged["target_5"].isna()
# X = df_lagged[["variable"]+num_cols]#.copy() 
# y = df_lagged["target_5"] ## target in 5Y

X = df_lagged[mask][["variable"]+num_cols]#.copy() 
y = df_lagged[mask]["target_5"] ## target in 5Y
assert (X.shape[0]>100)
assert (y.isna().sum()==0)

In [28]:
X

,variable,reviews,patent_count,patent_frac,patent_yearly_total,year_num,y_raw,lag5_pct_new,pct_diff_lag5,research_review_diff,research_review_ratio,pct_diff,review_research_ratio_2,review_research_diff_2,pat_div_research,y_diff,y_pct_diff
0,BLAST,1.4179,87.0,4.5645,1906.0,1979,103.5087,60.963088,0.182154,103.5087,1.0000,-0.0271,0.0137,-102.0908,0.8405,-2.8860,-0.027125
1,Brodmann area,0.0000,0.0,0.0000,1906.0,1979,2.1269,80.003761,NaN,-4.2538,-2.0000,0.9324,0.0000,-2.1269,0.0000,1.0263,0.932491
2,Cingulate cortex,0.0000,0.0,0.0000,1906.0,1979,13.8248,-16.910190,-0.221751,13.8248,1.0000,-0.0084,0.0000,-13.8248,0.0000,-0.1166,-0.008364
3,DNA array,0.3545,0.0,0.0000,1906.0,1979,4.2538,80.001411,NaN,4.2538,1.0000,0.1595,0.0833,-3.8993,0.0000,0.5850,0.159453
4,Dorsolateral prefrontal cortex,0.0000,0.0,0.0000,1906.0,1979,1.0634,60.005642,NaN,-40.0564,-37.6667,-0.5859,0.0000,-1.0634,0.0000,-1.5047,-0.585920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170,vaccine,258.3001,299.0,1.5224,19640.0,2015,1467.2528,-2.330014,-0.003188,1404.8250,0.9575,-0.0002,0.1760,-1208.9527,0.2038,-0.3561,-0.000243
4171,viral therapy,122.8675,16.0,0.0815,19640.0,2015,588.2531,-5.855541,0.023119,537.3565,0.9135,-0.0271,0.2089,-465.3856,0.0272,-16.3785,-0.027088
4172,vitamin C,12.7241,0.0,0.0000,19640.0,2015,188.3969,-12.409971,0.011673,183.2277,0.9726,-0.0447,0.0675,-175.6728,0.0000,-8.8053,-0.044651
4173,vitamin D,69.2671,0.0,0.0000,19640.0,2015,393.6532,14.166149,0.147495,385.7801,0.9800,0.0023,0.1760,-324.3861,0.0000,0.9081,0.002312


In [29]:
# %%time
# if GET_DL_EMBEDS:
#     print("Pre embed model (with all numeric features)")

# X2 = X.drop(columns=["variable"]).copy()
# model = CatBoostRegressor(verbose=False,has_time=True)

# y_pred_base = cross_val_predict(model,X2,y,groups=X["variable"],cv=GroupKFold(30))
# assert y_pred_base.shape[0]>4
# regression_results(y, y_pred_base)



# Pre embed  model (with all numeric features)
# r2: 0.866
# MAE (Mean absolute error): 134.453
# Median absolute error: 33.697
# RMSE: 507.23
# explained_variance:  0.867
# mape: 1.16

In [31]:
%%time
print("linear model with base feats") ## note - this fillna is leaky
X2 = X.select_dtypes("number").fillna(X.select_dtypes("number").mean()).copy()#.dropna(how="any",axis=1)
model = RidgeCV()
y_pred_base = cross_val_predict(model,X2,y,groups=X["variable"],cv=GroupKFold(30))
assert y_pred_base.shape[0]>4
regression_results(y, y_pred_base)


linear model with base feats
r2: 0.977
MAE (Mean absolute error): 99.044
Median absolute error: 48.264
RMSE: 209.82
explained_variance:  0.977
mape: 4.217
CPU times: user 1.27 s, sys: 45 ms, total: 1.32 s
Wall time: 185 ms


In [13]:
## frequently crashes when running on cpu, randomly. # but -  it improves model perf!! 
if GET_DL_EMBEDS:
    import gc
    import torch
    gc.collect()
    torch.cuda.empty_cache()
    from sentence_transformers import SentenceTransformer
    torch.cuda.empty_cache()
    model = SentenceTransformer('all-MiniLM-L12-v2',device="cpu") # same res with L6
    #Our sentences we like to encode
    sentences = list(df_lagged["variable"].unique())
    print(len(sentences))

    #Sentences are encoded by calling model.encode()
    embeddings = model.encode(sentences,batch_size=64,show_progress_bar=True,)#.to("cpu")

    df_embed_feats = pd.DataFrame(index=sentences, data=embeddings)
    print(df_embed_feats.shape)
    df_embed_feats.columns = ["embed_"+str(i) for i in df_embed_feats.columns]
#     df_embed_feats.to_parquet("var_text_embeddings.parquet")
    
#     ### OPT - truncate size of dims
# #     ### Has SAME Results as using all embeds
    pca = PCA()#64)
    pca.fit(df_embed_feats)
    print(pca.explained_variance_ratio_[0:2])
    print(sum(pca.explained_variance_ratio_))
    print(len(pca.singular_values_))
    df_embed_feats = pd.DataFrame(pca.transform(df_embed_feats),index=df_embed_feats.index)
    df_embed_feats.columns = ["embed_"+str(i) for i in df_embed_feats.columns]

    print(X.shape)
    X = X.join(df_embed_feats,on="variable",how="left")
#     df_lagged = df_lagged.join(df_embed_feats,on="variable",how="left")
#     ### our prediction code assumes "pred" to be the last column - resort this - hack
#     temp = df_lagged["preds"]
#     df_lagged.drop(columns=["preds"],inplace=True)
#     df_lagged["preds"] = temp
    print(X.shape)

2023-07-01 14:38:25.435431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


123


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

(123, 384)
[0.0721941  0.04845909]
1.0000000108266303
123
(4175, 17)
(4175, 140)


In [14]:
print("Naive lag model")

X2 = X[["y_raw"]].copy()
model = LinearRegression()
y_pred_base = cross_val_predict(model,X2,y,groups=X["variable"],cv=GroupKFold(30))

assert y_pred_base.shape[0]>4
regression_results(y, y_pred_base)

Naive lag model
r2: 0.973
MAE (Mean absolute error): 109.769
Median absolute error: 40.18
RMSE: 227.891
explained_variance:  0.973
mape: 1.881
mean_squared_log_error: 0.648


In [15]:
%%time
print("linear model with all features")
X2 = X.select_dtypes("number").fillna(X.select_dtypes("number").mean()).copy()#.dropna(how="any",axis=1)
model = RidgeRegressionCV()
y_pred_base = cross_val_predict(model,X2,y,groups=X["variable"],cv=GroupKFold(30))
assert y_pred_base.shape[0]>4
regression_results(y, y_pred_base)


linear model with all features
r2: 0.919
MAE (Mean absolute error): 268.285
Median absolute error: 187.304
RMSE: 394.517
explained_variance:  0.919
mape: 16.56
CPU times: user 4.62 s, sys: 162 ms, total: 4.78 s
Wall time: 600 ms


In [16]:
%%time

X2 = X.drop(columns=["variable"]).copy()
model = CatBoostRegressor(#iterations=300,
#                            cat_features= ["variable"],
                          verbose=False,has_time=True)

y_pred_base = cross_val_predict(model,X2,y,groups=X["variable"],cv=GroupKFold(30))
assert y_pred_base.shape[0]>4
regression_results(y, y_pred_base)


r2: 0.748
MAE (Mean absolute error): 198.612
Median absolute error: 58.369
RMSE: 697.184
explained_variance:  0.752
mape: 2.942
CPU times: user 25min 50s, sys: 55.7 s, total: 26min 46s
Wall time: 2min 15s


In [17]:
X

,variable,reviews,patent_count,patent_frac,patent_yearly_total,year_num,y_raw,lag5_pct_new,pct_diff_lag5,research_review_diff,...,embed_113,embed_114,embed_115,embed_116,embed_117,embed_118,embed_119,embed_120,embed_121,embed_122
0,BLAST,1.4179,87.0,4.5645,1906.0,1979,103.5087,60.963088,0.182154,103.5087,...,-0.006487,-0.010811,0.032403,0.002036,0.005526,0.012911,-0.011583,-0.010811,9.126961e-08,2.793968e-09
1,Brodmann area,0.0000,0.0,0.0000,1906.0,1979,2.1269,80.003761,NaN,-4.2538,...,-0.004383,-0.012107,0.015040,0.004735,0.001150,-0.004449,0.001217,-0.007151,5.960464e-08,-1.192093e-07
2,Cingulate cortex,0.0000,0.0,0.0000,1906.0,1979,13.8248,-16.910190,-0.221751,13.8248,...,0.013775,-0.068941,-0.024898,-0.062252,-0.008663,0.019254,0.012983,-0.043695,6.332994e-08,3.911555e-08
3,DNA array,0.3545,0.0,0.0000,1906.0,1979,4.2538,80.001411,NaN,4.2538,...,0.014190,-0.002436,-0.011945,0.003362,-0.001497,-0.010873,-0.032636,0.003842,2.607703e-08,2.700835e-08
4,Dorsolateral prefrontal cortex,0.0000,0.0,0.0000,1906.0,1979,1.0634,60.005642,NaN,-40.0564,...,0.006156,0.037015,-0.060375,0.044702,0.008988,0.002008,0.021354,-0.092173,3.841706e-08,-5.215406e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170,vaccine,258.3001,299.0,1.5224,19640.0,2015,1467.2528,-2.330014,-0.003188,1404.8250,...,0.034437,0.041638,0.017947,0.026666,0.008839,0.002105,0.012692,0.009824,7.450581e-08,1.234002e-08
4171,viral therapy,122.8675,16.0,0.0815,19640.0,2015,588.2531,-5.855541,0.023119,537.3565,...,-0.008241,-0.030304,-0.016297,-0.004147,-0.022418,-0.078247,0.041659,0.005014,5.774200e-08,1.117587e-08
4172,vitamin C,12.7241,0.0,0.0000,19640.0,2015,188.3969,-12.409971,0.011673,183.2277,...,0.008407,0.016892,0.003395,0.025371,-0.003974,0.000054,0.004048,0.008300,4.470348e-08,2.048910e-08
4173,vitamin D,69.2671,0.0,0.0000,19640.0,2015,393.6532,14.166149,0.147495,385.7801,...,-0.002899,-0.005251,-0.002317,-0.017406,-0.002889,-0.002569,-0.004854,-0.004081,2.235174e-08,-3.818423e-08
